# SageMaker JumpStart 모델 온보딩 (Pytorch)



## ML 모델 패키징 프로세스

<img src="images/ml-model-publishing-workflow.png"/>

다음 다이어그램은 ML 모델 패키징 프로세스의 개요를 제공합니다.

- **1단계** 모델 아티팩트 및 서빙/스코어링 로직 저장
- **2단계** 추론을 수행하는 SageMaker에서 모델을 호스팅하는 데 사용되는 컨테이너를 생성하고 ECR에 푸시
- **3단계** SageMaker에서 모델을 성공적으로 호스팅할 수 있는 컨테이너 검증
- **4단계** ML 모델을 모델 패키지로 패키징
- **5단계** Amazon SageMaker에 배포하여 ML 모델 패키지 검증
- **6단계** AWS Marketplace에 ML 모델 등록

> **참고**: 모든 로컬 작업은 최적의 성능과 호환성을 위해 반드시 GPU가 지원되는 SageMaker Notebook 인스턴스에서 수행되어야 합니다.

In [ ]:
install_needed = True
# install_needed = False

In [ ]:
%%bash
#!/bin/bash

DAEMON_PATH="/etc/docker"
MEMORY_SIZE=10G

FLAG=$(cat $DAEMON_PATH/daemon.json | jq 'has("data-root")')
# echo $FLAG

if [ "$FLAG" == true ]; then
    echo "Already revised"
else
    echo "Add data-root and default-shm-size=$MEMORY_SIZE"
    sudo cp $DAEMON_PATH/daemon.json $DAEMON_PATH/daemon.json.bak
    sudo cat $DAEMON_PATH/daemon.json.bak | jq '. += {"data-root":"/home/ec2-user/SageMaker/.container/docker","default-shm-size":"'$MEMORY_SIZE'"}' | sudo tee $DAEMON_PATH/daemon.json > /dev/null
    sudo service docker restart
    echo "Docker Restart"
fi

sudo curl -L "https://github.com/docker/compose/releases/download/v2.7.0/docker-compose-$(uname -s)-$(uname -m)" -o /usr/local/bin/docker-compose
sudo chmod +x /usr/local/bin/docker-compose

In [ ]:
import sys
import IPython

if install_needed:
    print("installing deps and restarting kernel")
    !{sys.executable} -m pip install --upgrade pip --quiet
    !{sys.executable} -m pip install -U sagemaker --quiet
    !{sys.executable} -m pip install -U transformers --quiet
    IPython.Application.instance().kernel.do_shutdown(True)

# Start

In [ ]:
%load_ext autoreload
%autoreload 2

### Model Store

In [ ]:
import os
import sagemaker
from sagemaker import get_execution_role

role = get_execution_role()
os.environ['HF_HOME'] = '/home/ec2-user/SageMaker/.cache'

In [ ]:
from pathlib import Path

# model_name_or_path='Salesforce/blip2-opt-2.7b'
model_name_or_path='Salesforce/blip-image-captioning-large'
cache_dir = f'{Path.cwd()}/cache_dir'

In [ ]:
import jinja2
jinja_env = jinja2.Environment()

<br>

## [**Step 1**] 모델 아티팩트 및 서빙/스코어링 로직 저장
---

### 모델 준비

In [ ]:
import os

In [ ]:
import transformers

In [ ]:
def download_model(hf_model_id, local_model_path, ignore_patterns=None):
    import shutil
    from pathlib import Path
    from huggingface_hub import snapshot_download

    # 기본적으로 무시할 패턴 설정 (safetensors는 반드시 포함해야 함)
    if ignore_patterns is None:
        ignore_patterns = ["*.ckpt"]  # 큰 체크포인트 파일만 무시
    
    local_model_path = Path(local_model_path)
    print(f"Downloading model to: {local_model_path}")
    
    # 디렉토리 준비
    if os.path.exists(local_model_path):
        print(f"Model directory already exists. Updating files at: {local_model_path}")
    else:
        os.makedirs(local_model_path, exist_ok=True)
        print(f"Created new model directory: {local_model_path}")
    
    try:
        # 모델 다운로드
        allow_patterns = ["*.json", "*.safetensors", "*.pt", "*.txt", "*.model", "*.tiktoken", "*.gguf"]
        
        snapshot_download(
            repo_id=hf_model_id,
            local_dir=local_model_path,
            local_dir_use_symlinks=False,
            ignore_patterns=ignore_patterns,
            allow_patterns=allow_patterns,
        )
        
        # 다운로드 검증
        model_files = os.listdir(local_model_path)
        print(f"Downloaded files: {model_files}")
        
        # 필수 파일 확인
        if not os.path.exists(os.path.join(local_model_path, "config.json")):
            raise FileNotFoundError("config.json이 다운로드되지 않았습니다!")
        
        # 가중치 파일 확인
        weight_found = False
        for file in model_files:
            if file.endswith('.bin') or file.endswith('.safetensors'):
                weight_found = True
                break
                
        if not weight_found:
            raise FileNotFoundError("모델 가중치 파일이 다운로드되지 않았습니다!")
            
        print(f"모델 다운로드 완료: {hf_model_id}")
        return local_model_path
    
    except Exception as e:
        print(f"모델 다운로드 오류: {e}")
        raise

In [ ]:
!rm -rf serve/checkpoint

In [ ]:
model_name_or_path

In [ ]:
download_model(model_name_or_path, f"serve/checkpoint")

아래 코드 셀은 src 디렉토리에 SageMaker 추론 스크립트를 저장합니다.

#### Option 1.
- `model_fn(model_dir)`: S3의 `model_dir`에 저장된 모델 아티팩트를 로드합니다.
- `input_fn(request_body, content_type)`: 입력 데이터를 전처리합니다. `content_type`은 입력 데이터 종류에 따라 다양하게 처리 가능합니다. (예: `application/x-npy`, `application/json`, `application/csv`등)
- `predict_fn(input_object, model)`: `input_fn(...)`을 통해 들어온 데이터에 대해 추론을 수행합니다.
- `output_fn(prediction, accept_type)`: `predict_fn(...)`에서 받은 추론 결과를 후처리를 거쳐 프론트엔드로 전송합니다.

#### Option 2.
- `model_fn(model_dir)`: S3의 model_dir에 저장된 모델 아티팩트를 로드합니다.
- `transform_fn(model, request_body, content_type, accept_type)`: `input_fn(...), predict_fn(...), output_fn(...)`을 `transform_fn(...)`으로 통합할 수 있습니다.

In [ ]:
%%writefile serve/inference.py
import os
import json
import logging
import numpy as np
import torch

from io import BytesIO
import base64
from PIL import Image
import transformers

from typing import Any
from typing import Dict

from sagemaker_inference import encoder

data_type = torch.float16
device = "cuda" if torch.cuda.is_available() else "cpu"


def decode_image(request_body):
    buff = BytesIO(base64.b64decode(request_body))
    image = Image.open(buff)
    return image


def _validate_payload(payload: Dict[str, Any]) -> None:
    """Validate the parameters in the input loads.

    Checks if image exists.
    Args:
        payload: a decoded input payload (dictionary of input parameter and values)
    """

    for param_name in payload:
        if param_name not in ['input_image']:
            raise KeyError(
                f"Input payload contains an invalid key input_image."
            ) 


def model_fn(model_dir: str) -> transformers:
    
    print(f"************ model_dir : {model_dir}")
    
    model = transformers.BlipForConditionalGeneration.from_pretrained(
        f'{model_dir}/checkpoint/',
        torch_dtype=data_type
    ).to(f"{device}:0")
    model.eval()
    
    # model2 = transformers.BlipForConditionalGeneration.from_pretrained(
    #     f'{model_dir}/code/checkpoint',
    #     torch_dtype=data_type
    # ).to(f"{device}:3")
    model.eval()

    processor = transformers.AutoProcessor.from_pretrained(
        f'{model_dir}/checkpoint/'
    )
    torch.cuda.empty_cache()
    return (model, processor)


def transform_fn(model_return: transformers, payload: bytes, content_type: str, accept: str) -> str:
    torch.cuda.empty_cache()

    model, processor = model_return
    
    if content_type == "application/x-image":
        try:
            payload=decode_image(payload)
            inputs = processor(payload, return_tensors="pt").to((f"{device}:0"), data_type)
        except Exception:
            logging.exception(
                f"Failed to parse input payload. For content_type=application/x-image, input "
                f"payload must be a bytearray encoded image."
            )
            torch.cuda.empty_cache()
            raise
    elif content_type == "application/json":
        try:
            payload = json.loads(payload)
            _validate_payload(payload)
            payload=decode_image(payload['input_image'])
            inputs = processor(payload, return_tensors="pt").to((f"{device}:0"), data_type)
        except Exception:
            logging.exception(
                f"Failed to parse input payload. For content_type=application/json, input "
                f"payload must be a json encoded dictionary with keys input_image."
            )
            raise
    else:
        raise ValueError('{{"error": "unsupported content type {}"}}'.format(content_type or "unknown"))

    try:
        torch.cuda.empty_cache()
        out = model.generate(**inputs)
        caption_texts = processor.decode(out[0], skip_special_tokens=True)

        output = {
            "generated_caption" : caption_texts
        }
        torch.cuda.empty_cache()
    except torch.cuda.OutOfMemoryError as e:
        logging.error(
            "Model ran out of CUDA memory while generating images. Please reduce height and width or "
            f"deploy the model on an instance type with more GPU memory. Error: {e}."
        )
        torch.cuda.empty_cache()
        raise
    except Exception:
        logging.exception("Failed to caption images.")
        torch.cuda.empty_cache()
        raise
    return encoder.encode(output, accept)

<br>

## [**Step 2**] 추론 수행을 위한 SageMaker 컨테이너 생성 및 ECR 등록
---

In [ ]:
import boto3

region = boto3.Session().region_name

In [ ]:
image_uri = sagemaker.image_uris.retrieve(
    framework='pytorch',
    region=region,
    image_scope='inference',
    version='2.0',
    instance_type='ml.g5.12xlarge'
)
docker_account_id = image_uri.split('.')[0]
print(f'image_uri: {image_uri} \ndocker_account_id : {docker_account_id}')

In [ ]:
!rm -rf docker && mkdir docker

In [ ]:
%%writefile docker/Dockerfile

FROM 763104351884.dkr.ecr.us-west-2.amazonaws.com/pytorch-inference:2.0-gpu-py310

ENV TZ=Asia/Seoul
RUN ln -snf /usr/share/zoneinfo/$TZ /etc/localtime && echo $TZ > /etc/timezone

RUN pip install --no-cache-dir numpy==1.24.3

RUN pip install -U --no-cache-dir pip \
    easyocr \ 
    simplejpeg \
    Pillow

RUN pip install --no-cache-dir pytorch-lightning==2.0.1.post0 \
    git+https://github.com/openai/CLIP.git \
    git+https://github.com/fbcotter/pytorch_wavelets.git \
    git+https://github.com/richzhang/PerceptualSimilarity.git

RUN pip install --no-deps open_clip_torch==2.20.0
RUN pip install --no-cache-dir transformers==4.32.0 safetensors

ENV PYTHONUNBUFFERED=TRUE
ENV LANG C.UTF-8

In [ ]:
!rm -rf shell && mkdir shell

In [ ]:
%%writefile shell/build_and_push.sh

# The name of our algorithm
algorithm_name={{algorithm_name}}
image_tag={{image_tag}}

cd {{root_dir}}/docker

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-us-west-2}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:${image_tag}"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

aws ecr get-login-password --region ${region} | docker login --username AWS --password-stdin "{{docker_account_id}}.dkr.ecr.${region}.amazonaws.com"

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build -f Dockerfile -t ${fullname} .

# Get the login command from ECR and execute it directly
aws ecr get-login-password --region ${region}|docker login --username AWS --password-stdin ${fullname}

docker push ${fullname}

In [ ]:
algorithm_name='js-on-boarding-customer'
image_tag='blip'

In [ ]:
template = jinja_env.from_string(Path("shell/build_and_push.sh").open().read())
Path("shell/build_and_push.sh").open("w").write(template.render(algorithm_name=algorithm_name, image_tag=image_tag, root_dir=os.getcwd(), docker_account_id=docker_account_id))
# !pygmentize shell/build_and_push.sh | cat -n
!chmod +x shell/build_and_push.sh

In [ ]:
! ./shell/build_and_push.sh

In [ ]:
account = sagemaker.Session().account_id()
region = sagemaker.Session().boto_region_name

In [ ]:
ecr_image_uri=f"{account}.dkr.ecr.{region}.amazonaws.com/{algorithm_name}:{image_tag}"
ecr_image_uri

<br>

## [**Step 3**] SageMaker에서 모델을 성공적으로 호스팅할 수 있는 컨테이너 검증
---

SageMaker 호스팅 엔드포인트로 배포하기 전에 로컬 모드 엔드포인트로 배포할 수 있습니다. 로컬 모드는 현재 개발 중인 환경에서 도커 컨테이너를 실행하여 SageMaker 프로세싱/훈련/추론 작업을 에뮬레이트할 수 있습니다. 추론 작업의 경우는 Amazon ECR의 딥러닝 프레임워크 기반 추론 컨테이너를 로컬로 가져오고(docker pull) 컨테이너를 실행하여(docker run) 모델 서버를 시작합니다.


### Uploading Model Data to S3

In [ ]:
sagemaker_session = sagemaker.Session()

In [ ]:
bucket=sagemaker_session.default_bucket()
prefix='blip-large/model_data_customers'

In [ ]:
!sudo rm -rf serve/.ipynb_checkpoints/
!sudo rm -rf serve/__pycache__/

In [ ]:
model_root_path = Path("./serve")
model_upload_paths = {}

tar_name = "model.tar.gz"

!tar -C $model_root_path -czvf $tar_name checkpoint inference.py
sagemaker_session.upload_data(path=tar_name, bucket=bucket, key_prefix=prefix)
!sudo rm -rf ./model && mkdir ./model
!mv $tar_name ./model/

In [ ]:
model_data_url = f's3://{bucket}/{prefix}/{tar_name}'
model_data_url

### SageMaker Endpoint (Local Mode)

로컬 모드는 필수로 수행할 필요는 없지만, 디버깅에 많은 도움이 됩니다. 또한, 로컬 모드 사용 시에는 모델을 S3에 반드시 업로드할 필요 없이 로컬 디렉터리에서도 로드할 수 있습니다. (`container` 변수 참조)

In [ ]:
import boto3
import time

# Set to True to enable SageMaker to run locally
local_mode = True
# local_mode = False
if local_mode:
    from sagemaker.local import LocalSession
    instance_type = "local_gpu"
    sm_session = LocalSession()
    sm_session.config = {'local': {'local_code': True}}
    sm_client = sagemaker.local.LocalSagemakerClient()
    smr_client = sagemaker.local.LocalSagemakerRuntimeClient()
    model_data=f"file://{Path.cwd()}/model/model.tar.gz"
else:
    instance_type = "ml.g5.2xlarge"
    sm_session = sagemaker.Session()
    sm_client = boto3.client("sagemaker")
    smr_client = boto3.client("sagemaker-runtime")
    model_data = model_data_url

instance_count = 1
ts = time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())
sm_model_name = f"blip-model-{ts}"
endpoint_config_name = f"blip-endpoint-config-{ts}"
endpoint_name = f"blip-endpoint-{ts}"
model_data

In [ ]:
env = {
    "SAGEMAKER_CONTAINER_LOG_LEVEL" : "20",
    "SAGEMAKER_PROGRAM" : "inference.py",
    "SAGEMAKER_REGION" : "us-west-2"
}

container = {
    "Image": ecr_image_uri,
    "ModelDataUrl": model_data,
    "Environment": env
}

In [ ]:
create_model_response = sm_client.create_model(
    ModelName=sm_model_name, 
    ExecutionRoleArn=role, 
    PrimaryContainer=container,
)

create_endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "InstanceType": instance_type,
            "InitialVariantWeight": 1,
            "InitialInstanceCount": 1,
            "ModelName": sm_model_name,
            "VariantName": "AllTraffic",
        }
    ],
)
#print("Model Arn: " + create_model_response["ModelArn"])

In [ ]:
!docker ps

In [ ]:
!docker kill 8af518fd8b56

In [ ]:
create_endpoint_response = sm_client.create_endpoint(
    EndpointName=endpoint_name, EndpointConfigName=endpoint_config_name
)

In [ ]:
!docker ps

### Inference Test

In [ ]:
import base64
from PIL import Image
from io import BytesIO

def encode_image(image):
    buffer = BytesIO()
    image.save(buffer, format="JPEG")
    img_str = base64.b64encode(buffer.getvalue())
    return img_str


# def decode_image(img):
#     img = img.encode("utf8") if type(img) == "bytes" else img
#     buff = BytesIO(base64.b64decode(img))
#     image = Image.open(buff)
#     return image

In [ ]:
import requests
import numpy as np

url = 'https://media.newyorker.com/cartoons/63dc6847be24a6a76d90eb99/master/w_1160,c_limit/230213_a26611_838.jpg'
image = Image.open(requests.get(url, stream=True).raw).convert('RGB')  
display(image.resize((596, 437)))
input_image = encode_image(image)

In [ ]:
def invoke_endpoint(endpoint_name, payload):
    import json
    
    response = smr_client.invoke_endpoint(
        EndpointName=endpoint_name,
        Accept="application/json",
        ContentType="application/x-image",
        Body=payload
    )
    data = response["Body"].read()
    output = json.loads(data)
    return output



def invoke_endpoint_json(endpoint_name, payload, target_model=None):
    import json
    
    response = smr_client.invoke_endpoint(
        EndpointName=endpoint_name,
        Accept="application/json",
        ContentType="application/json",
        Body=json.dumps(payload)
    )
    data = response["Body"].read()
    output = json.loads(data)
    return output

In [ ]:
res = invoke_endpoint(endpoint_name, input_image)
res

In [ ]:
input_value = {'input_image' : input_image.decode("utf-8")}
res = invoke_endpoint_json(endpoint_name, input_value)
res

In [ ]:
def delete_endpoint(client, endpoint_name):
    response = client.describe_endpoint(EndpointName=endpoint_name)
    EndpointConfigName = response['EndpointConfigName']
    
    response = client.describe_endpoint_config(EndpointConfigName=EndpointConfigName)
    model_name = response['ProductionVariants'][0]['ModelName']
    
    client.delete_model(ModelName=model_name)    
    client.delete_endpoint_config(EndpointConfigName=EndpointConfigName) 
    client.delete_endpoint(EndpointName=endpoint_name)
   
    print(f'--- Deleted model: {model_name}')
    print(f'--- Deleted endpoint_config: {EndpointConfigName}')     
    print(f'--- Deleted endpoint: {endpoint_name}')

In [ ]:
delete_endpoint(sm_client, endpoint_name)
!sudo rm -rf /tmp/tmp*

### SageMaker Endpoint 에서 확인 (원격에서 호스팅)
> **⚠️ 주의** : 테스트 후 **반드시** 인스턴스 삭제가 필요합니다. 콘솔에서 SageMakerAI - Inference 에 SageMaker Endpoint가 삭제되었는지 확인해 주세요.

In [ ]:
import boto3
import time

# Set to True to enable SageMaker to run locally
local_mode = False

if local_mode:
    from sagemaker.local import LocalSession
    instance_type = "local_gpu"
    sm_session = LocalSession()
    sm_session.config = {'local': {'local_code': True}}
    sm_client = sagemaker.local.LocalSagemakerClient()
    smr_client = sagemaker.local.LocalSagemakerRuntimeClient()
    model_data=f"file://{Path.cwd()}/model/model.tar.gz"
else:
    instance_type = "ml.g5.2xlarge"
    sm_session = sagemaker.Session()
    sm_client = boto3.client("sagemaker")
    smr_client = boto3.client("sagemaker-runtime")
    model_data = model_data_url

instance_count = 1
ts = time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())
sm_model_name = f"blip-model-{ts}"
endpoint_config_name = f"blip-endpoint-config-{ts}"
endpoint_name = f"blip-endpoint-{ts}"
model_data

In [ ]:
def invoke_endpoint(endpoint_name, payload):
    import json
    
    response = smr_client.invoke_endpoint(
        EndpointName=endpoint_name,
        Accept="application/json",
        ContentType="application/x-image",
        Body=payload
    )
    data = response["Body"].read()
    output = json.loads(data)
    return output



def invoke_endpoint_json(endpoint_name, payload, target_model=None):
    import json
    
    response = smr_client.invoke_endpoint(
        EndpointName=endpoint_name,
        Accept="application/json",
        ContentType="application/json",
        Body=json.dumps(payload)
    )
    data = response["Body"].read()
    output = json.loads(data)
    return output

In [ ]:
env = {
    "SAGEMAKER_CONTAINER_LOG_LEVEL" : "20",
    "SAGEMAKER_PROGRAM" : "inference.py",
    "SAGEMAKER_REGION" : "us-west-2",
}

# model_data_url = f"s3://{bucket}/{prefix}/"  # s3 location where models are stored

container = {
    "Image": ecr_image_uri,
    "ModelDataUrl": model_data,
    "Environment": env
}
container

In [ ]:
create_model_response = sm_client.create_model(
    ModelName=sm_model_name, ExecutionRoleArn=role, PrimaryContainer=container
)

create_endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "InstanceType": instance_type,
            "InitialVariantWeight": 1,
            "InitialInstanceCount": 1,
            "ModelName": sm_model_name,
            "VariantName": "AllTraffic",
        }
    ],
)
print("Model Arn: " + create_model_response["ModelArn"])
print("Endpoint Config Arn: " + create_endpoint_config_response["EndpointConfigArn"])

In [ ]:
create_endpoint_response = sm_client.create_endpoint(
    EndpointName=endpoint_name, EndpointConfigName=endpoint_config_name
)

print("Endpoint Arn: " + create_endpoint_response["EndpointArn"])

In [ ]:
from IPython.display import display, HTML
def make_console_link(region, endpoint_name, task='[SageMaker LLM Serving]'):
    endpoint_link = f'<b> {task} <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={region}#/endpoints/{endpoint_name}">Check Endpoint Status</a></b>'   
    return endpoint_link

endpoint_link = make_console_link(region, endpoint_name)
display(HTML(endpoint_link))

In [ ]:
resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp["EndpointStatus"]
print("Status: " + status)

while status == "Creating":
    time.sleep(30)
    resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
    status = resp["EndpointStatus"]
    print("Status: " + status)

print("Arn: " + resp["EndpointArn"])
print("Status: " + status)

In [ ]:
response = smr_client.invoke_endpoint(
    EndpointName=endpoint_name,
    Accept="application/json",
    ContentType="application/x-image",
    Body=input_image
)

In [ ]:
%time
res=invoke_endpoint(endpoint_name, input_image)
res

In [ ]:
%time
input_value = {'input_image' : input_image.decode("utf-8")}
res = invoke_endpoint_json(endpoint_name, input_value)
res

### Clean up

In [ ]:
def delete_endpoint(client, endpoint_name):
    response = client.describe_endpoint(EndpointName=endpoint_name)
    EndpointConfigName = response['EndpointConfigName']
    
    response = client.describe_endpoint_config(EndpointConfigName=EndpointConfigName)
    model_name = response['ProductionVariants'][0]['ModelName']
    
    client.delete_model(ModelName=model_name)    
    client.delete_endpoint_config(EndpointConfigName=EndpointConfigName) 
    client.delete_endpoint(EndpointName=endpoint_name)
   
    print(f'--- Deleted model: {model_name}')
    print(f'--- Deleted endpoint_config: {EndpointConfigName}')     
    print(f'--- Deleted endpoint: {endpoint_name}')

In [ ]:
delete_endpoint(sm_client, endpoint_name)

<br>

## [**Step 4**] ML 모델을 모델 패키지로 패키징
---
이 **step**에서는 아티팩트(ECR 이미지 및 학습된 모델 아티팩트)를 ModelPackage로 패키징하는 방법을 살펴봅니다. 이 작업을 완료하면 AWS 마켓플레이스에서 제품을 사전 학습된 모델로 등록할 수 있습니다.

**Note:** 모델을 여러 하드웨어 유형(CPU/GPU/Inferentia)에 배포할 수 있는 경우, 일반적으로 사용되는 컨테이너 이미지가 각각 다르기 때문에 각각에 대해 모델패키지를 생성하고 MP 목록에 다른 버전으로 추가해야 합니다.  

### 모델 패키지 사전 준비
모델 패키지는 추론에 필요한 모든 요소를 패키지로 묶은 모델 아티팩트에 대한 재사용 가능한 추상화 형태입니다. 이는 모델 데이터 위치(선택 사항)와 함께 사용할 추론 이미지를 정의하는 추론 사양으로 구성됩니다. ModelPackage는 AWS 마켓플레이스에 판매자로 등록할 AWS 계정에서 생성해야 합니다.

In [ ]:
import os
import boto3
import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()
prefix='blip-large/model_data_customers'

region = sagemaker_session.boto_region_name
account= boto3.client("sts").get_caller_identity().get("Account")
role = get_execution_role()

s3_client = sagemaker_session.boto_session.client("s3")
sm_runtime = boto3.client("sagemaker-runtime")

In [ ]:
model_data = f's3://{bucket}/{prefix}/model.tar.gz'
model_data

In [ ]:
algorithm_name='js-on-boarding-customer'
image_tag='blip'

In [ ]:
ecr_image_uri=f"{account}.dkr.ecr.{region}.amazonaws.com/{algorithm_name}:{image_tag}"
ecr_image_uri

In [ ]:
# <<YourSupportedContentTypes>>
supported_content_types = ["application/x-image", "application/json"] #["text/csv", "application/json", "application/json", "application/jsonlines"]

# <<YourSupportedResponseMIMETypes>>
supported_response_MIME_types = [ 
    "application/json",
    # "text/csv",
    # "application/jsonlines",
]

supported_realtime_inference_instance_types = ["ml.g5.2xlarge"]
supported_batch_transform_instance_types = ["ml.g5.xlarge"] #  Don't use batch transform. And, the Batch Transform validation step is not required

### 모델 패키지 생성
모델 패키지 생성 프로세스에서는 다음을 지정해야 합니다:
  1. 도커 이미지
  2. 모델 아티팩트
    - tar.gz 형태로 압축된 모델 아티팩트가 제공되어야 합니다.
        
판매자(및 구매자)에게 Amazon SageMaker에서 제품이 작동한다는 확신을 주기 위해, AWS Marketplace에 제품을 리스팅하기 전에 SageMaker는 기본적인 유효성 검사를 위와 같이 진행하였습니다. 이 유효성 검사 프로세스가 성공해야만 제품을 AWS Marketplace에 리스팅할 수 있습니다. 이 유효성 검사 프로세스는 사용자가 제공한 유효성 검사 프로필과 샘플 데이터를 사용하여 모델을 사용하여 계정에서 변환 작업을 생성하여 추론 이미지가 SageMaker에서 작동하는지 확인합니다.

다음으로, ML 모델에 적합한 인스턴스 크기를 식별해야 하며, ML 모델 위에서 성능 테스트를 실행하여 이를 확인할 수 있습니다.

**Note:** 모델 튜닝 외에도 인스턴스 유형을 식별할 때 모델의 요구 사항을 고려해야 합니다.  모델이 GPU 리소스를 사용하지 않는 경우 GPU 인스턴스 유형을 포함하지 마세요. 마찬가지로 모델이 GPU 리소스를 사용하지만 단일 GPU만 사용할 수 있는 경우, 여러 개의 GPU가 있는 인스턴스 유형을 포함하지 마세요. 성능상의 이점은 없이 사용자의 인프라 요금만 증가시킬 수 있기 때문입니다.

### 테스트용 샘플이미지 만들기

In [ ]:
import base64
from PIL import Image
from io import BytesIO

def encode_image(image):
    buffer = BytesIO()
    image.save(buffer, format="JPEG")
    img_str = base64.b64encode(buffer.getvalue())
    return img_str

In [ ]:
import requests
import numpy as np

url = 'https://media.newyorker.com/cartoons/63dc6847be24a6a76d90eb99/master/w_1160,c_limit/230213_a26611_838.jpg'
image = Image.open(requests.get(url, stream=True).raw).convert('RGB')  
display(image.resize((596, 437)))
input_image = encode_image(image)

In [ ]:
payload = {'input_image' : input_image.decode("utf-8")}

In [ ]:
from sagemaker.session import Session

sagemaker_session = Session()
s3_client = sagemaker_session.boto_session.client("s3")

In [ ]:
bucket = sagemaker_session.default_bucket()

In [ ]:
import json
json_line = json.dumps(payload)
with open("input.jsonl", "w") as f:
    f.write(json_line)
s3_client.put_object(Bucket=bucket, Key="validation-input-json/input.jsonl", Body=json_line)

In [ ]:
validation_file_name = "input.jsonl"
validation_input_path = f"s3://{bucket}/validation-input-json/"
validation_output_path = f"s3://{bucket}/validation-output-jsonl/"

### 패키지 생성

In [ ]:
# Define parameters
model_name = "marketplace-model-test-1" #"<<YourModelName>>"
model_description = "marketplace-model-test" #"<<YourModelDescription>>"

In [ ]:
model_package = sagemaker_session.sagemaker_client.create_model_package(
    ModelPackageName=model_name,
    ModelPackageDescription=model_description,
    InferenceSpecification={
        "Containers": [
            {
                "Image": ecr_image_uri,
                "ModelDataUrl": model_data
            }
        ],
        "SupportedTransformInstanceTypes": supported_batch_transform_instance_types,
        "SupportedRealtimeInferenceInstanceTypes": supported_realtime_inference_instance_types,
        "SupportedContentTypes": supported_content_types,
        "SupportedResponseMIMETypes": supported_response_MIME_types,
    },
    CertifyForMarketplace=True,  # Make sure to set this to True
   ValidationSpecification={
        'ValidationRole': role,
        'ValidationProfiles': [
            {
                'ProfileName': "validation",
                'TransformJobDefinition': {
                    'MaxConcurrentTransforms': 1,
                    'MaxPayloadInMB': 64,
                    'BatchStrategy': 'SingleRecord',
                    'TransformInput': {
                        'DataSource': {
                            'S3DataSource': {
                                'S3DataType': 'S3Prefix',
                                'S3Uri': f'{validation_input_path}input.jsonl'
                            }
                        },
                        'ContentType': 'application/json',
                        'CompressionType': 'None',
                        'SplitType': 'None'
                    },
                    'TransformOutput': {
                        'S3OutputPath': f'{validation_output_path}output.json',
                        'Accept': 'application/json',
                        'AssembleWith': 'None',
                    },
                    'TransformResources': {
                        'InstanceType': 'ml.g5.xlarge',
                        'InstanceCount': 1,
                    }
                }
            },
        ]
    },
)

In [ ]:
sagemaker_session.wait_for_model_package(model_package_name=model_name) # If failure occurs navigate to SageMaker Console > My marketplace model packages > select the failed ModelPackage for details. 

다음을 실행하기 전에, [Model Packages console from Amazon SageMaker](https://console.aws.amazon.com/sagemaker/home?region=us-east-1#/model-packages/my-resources)을 열어서 모델 생성의 성공했는지를 확인해야 합니다.
모델을 선택하고 **Validation** 탭을 열어서 validation 결과를 확인할 수 있습니다.

<br>

## [**Step 5**] Amazon SageMaker에 배포하여 ML 모델 패키지 검증
---

##### 모델 패키지에서 모델 객체 생성

In [ ]:
model_package['ModelPackageArn']

In [ ]:
from sagemaker import ModelPackage

model = ModelPackage(
    role=role,
    model_package_arn=model_package["ModelPackageArn"],
    sagemaker_session=sagemaker_session,
)

#### SageMaker 모델을 Endpoint로 배포

In [ ]:
model.deploy(
    initial_instance_count=1,
    instance_type=supported_realtime_inference_instance_types[0],
    endpoint_name=model_name,
)
model.endpoint_name

#### boto3로 예시 호출

In [ ]:
# Make use of your own example input data to test the Endpoint
#input_json = '{"text": "sample"}'

payload = {'input_image' : input_image.decode("utf-8")}

response = sm_runtime.invoke_endpoint(
    EndpointName=model.endpoint_name,
    ContentType="application/json",
    Accept="application/json",
    Body=json.dumps(payload),
)

json.load(response["Body"])

#### AWS CLI로 예시 호출

In [ ]:
sagemaker_session.boto_region_name

In [ ]:
# Perform inference
!aws sagemaker-runtime invoke-endpoint \
    --endpoint-name $model.endpoint_name \
    --body fileb://$validation_file_name \
    --content-type application/json \
    --region $sagemaker_session.boto_region_name \
    out.out
    
    
# Print inference
!head out.out

#### 생성된 endpoint configuration 과 endpoint 정리 

In [ ]:
model.sagemaker_session.delete_endpoint(model.endpoint_name)
model.sagemaker_session.delete_endpoint_config(model.endpoint_name)

- 이 모델은 필수가 아니므로 삭제해도 됩니다. 
- 배포 가능한 모델을 삭제한다는 점에 유의하세요. 
- 모델 패키지는 삭제하지 않습니다.

In [ ]:
model.delete_model()

##### AWS 마켓플레이스에 모델을 게시하려면 모델 패키지 ARN을 지정해야 합니다. 다음 모델 패키지 ARN을 복사합니다. 

In [ ]:
model_package["ModelPackageArn"]

<br>


## [**Step 6**] AWS Marketplace에 ML 모델 등록
---

1.  모델 파트너는 AWS 마켓플레이스에서 [public profile](https://docs.aws.amazon.com/marketplace/latest/userguide/seller-registration-process.html#seller-public-profile)을 생성하고 seller로 등록합니다.
마켓플레이스의 상품은 무료 상품으로 등록되므로 세금 정보를 제공할 필요가 없습니다.

2. 세이지메이커 콘솔의 [Model Packages](https://console.aws.amazon.com/sagemaker/home?region=us-east-1#/model-packages/my-resources) 섹션에서 이 노트북에서 생성한 엔티티를 찾을 수 있습니다. 성공적으로 생성되고 유효성이 검사되었다면 해당 엔티티를 선택하고 **Publish new ML Marketplace listing**를 선택할 수 있을 것입니다.

<img src="images/publish-to-marketplace-action.png"/>

리스팅을 작성할 수 있는 [AWS Marketplace Management portal](https://aws.amazon.com/marketplace/management/ml-products/)로 리디렉션됩니다.

<img src="images/listing.png"/>

1. 모델이 여러 하드웨어 유형을 대상으로 하는 경우 각 ModelPackage를 별도의 버전으로 목록에 추가하는 것을 잊지 마세요.
2. 추가를 클릭하고 모델 정보를 입력합니다. Product visibility을 'Public'로 설정해야 합니다.

<img src="images/public.png"/>

3. 테스트를 진행할 account 에 대해 모델 접근을 위한 Allowlist에 추가합니다. 예) account `171503325295`, `572320329544` and `559110549532` for access to the model. 
For region support select: `us-east-1, us-west-2, eu-west-1, eu-central-1, eu-west-2, ap-northeast-1, ap-south-1, ca-central-1, us-east-2, ap-northeast-2`
<img src="images/allowlist-accs.png"/>

4. Pricing and terms 하에 pricing 모델을 설정합니다.
**Inference based pricing (custom metering) at $0**

(선택 사항) 컨테이너가 아래를 구현하지 않은 경우 이를 확인하고 다음을 진행하세요. 

```
I confirm that my model package supports the response header for custom metering. Example response header: X-Amzn-Inference-Metering:
{"Dimension": "inference.count", "ConsumedUnits": 3}
I understand that in absence of this header, default metering will be used instead.
```

<img src="images/inference-based-pricing.png"/>

5. Listing 상태는 다음과 같이 표시되어야 합니다:
**Do not click Sign off and publish**

<img src="images/status-1.png"/>

6. Vissibility status of the listing should be `Limited`.

<img src="images/status-2.png"/>




**Resources**
* [Publishing your product in AWS Marketplace](https://docs.aws.amazon.com/marketplace/latest/userguide/ml-publishing-your-product-in-aws-marketplace.html)
